In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Configuracion de visualización de tablas en VS Code -Jupyter
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

In [4]:
# Lectura de los datos
movie_data = pd.read_csv('../datasets/movies_dataset.csv', low_memory=True)
movie_data.head(2)

C:\Users\Roy Quillca Pacco\AppData\Local\Temp\ipykernel_2768\1211225962.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_data = pd.read_csv('../datasets/movies_dataset.csv', low_memory=True)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


## 1. Diccionario de datos

1. **``adult``**: *Indica si la película tiene califiación X, exclusiva para adultos..*
2. **``belongs_to_collection``**: *Un diccionario que indica a que franquicia o serie de películas pertenece la película.*
3. **``budget``**: *El presupuesto de la película, en dólares.*
4. **``genres``**: *Un diccionario que indica todos los géneros asociados a la película.*
5. **``homepage``**: *La página web oficial de la película.*
6. **``id``**: *ID de la pelicula.*
7. **``imdb_id``**: *IMDB ID de la pelicula.*
8. **``original_language``**: *Idioma original en la que se grabo la pelicula.*
9. **``original_title``**: *Titulo original de la pelicula.*
10. **``overview``**: *Pequeño resumen de la película.*
11. **``popularity``**: *Puntaje de popularidad de la película, asignado por TMDB (TheMoviesDataBase).*
12. **``poster_path``**: *URL del póster de la película.*
13. **``production_companies``**: *Lista con las compañias productoras asociadas a la película.*
14. **``production_countries``**: *Lista con los países donde se produjo la película.*
15. **``release_date``**: *Fecha de estreno de la película.*
16. **``revenue``**: *Recaudación de la pelicula, en dolares.*
17. **``runtime``**: *Duración de la película, en minutos.*
18. **``spoken_languages``**: *Lista con los idiomas que se hablan en la pelicula.*
19. **``status``**: *Estado de la pelicula actual (si fue anunciada, si ya se estreno, etc).*
20. **``tagline``**: *Frase celebre asociadaa la pelicula.*
21. **``title``**: *Titulo de la pelicula.*
22. **``video``**: *Indica si hay o no un trailer en video disponible en TMDB.*
23. **``vote_average``**: *Puntaje promedio de reseñas de la pelicula.*
24. **``vote_count``**: *Numeros de votos recibidos por la pelicula, en TMDB.*


## **Transformaciones**

Para este MVP no necesitas perfección, ¡necesitas rapidez! ⏩ Vas a hacer estas, ***y solo estas***, transformaciones a los datos:

##### 1. Algunos campos, como **`belongs_to_collection`**, **`production_companies`** y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder  y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

*Crearemos una nueva columna ``collection_id``, ``collection_name``, ``collection_poster_path``, ``collection_backdrop_path``.*

In [5]:
import re

In [6]:
def extract_values(text):
    if type(text) == str:
        pattern = r"'(\w+)': '([^']+)'"
        matches = re.findall(pattern, text)
        return {key: value for key, value in matches}
    else:
        return None

In [7]:
# Función para extraer el valor del campo 'id'
def extract_id(text):
    if type(text) == str:
        pattern = r"'id': (\d+)"
        match = re.search(pattern, text)
        if match:
            return match.group(1)
        else:
            return None
    else:
        return None

In [8]:
# Aplicar la función a la columna y crear una nueva columna 'id'
movie_data['collection_id'] = movie_data['belongs_to_collection'].apply(extract_id)
# Aplicar la función a la columna y crear nuevas columnas en el DataFrame
movie_data[['collection_name', 'collection_poster_path', 'collection_backdrop_path']] = movie_data['belongs_to_collection'].apply(extract_values).apply(pd.Series)

In [9]:
# Eliminación de la columna belongs_to_collection
movie_data = movie_data.drop(axis="columns", columns="belongs_to_collection")

In [10]:
# Aplicar la función a la columna y crear una nueva columna 'production_company_id'
movie_data["production_company_id"] = movie_data["production_companies"].apply(extract_id)
# Aplicar la función a la columna y crear nuevas columnas en el DataFrame
movie_data["production_company_name"] = movie_data["production_companies"].apply(extract_values).apply(pd.Series)

In [11]:
# Eliminación de la columna belongs_to_collection
movie_data = movie_data.drop(axis="columns", columns="production_companies")

In [12]:
movie_data.columns

Index(['adult', 'budget', 'genres', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_countries', 'release_date', 'revenue',
       'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'collection_id', 'collection_name',
       'collection_poster_path', 'collection_backdrop_path',
       'production_company_id', 'production_company_name'],
      dtype='object')

##### 2. Los valores nulos de los campos **`revenue`**, **`budget`** deben ser rellenados por el número **`0`**.

In [13]:
movie_data[movie_data["revenue"].isna()].iloc[:10,10:20]

,poster_path,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video
19729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19730,Midnight Man,6.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Mardock Scramble: The Third Exhaust,7.0,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks,4.3,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Alerta de imputación masiva de valores nulos!!!

movie_data["revenue"] = movie_data["revenue"].fillna(value=0)
movie_data["budget"] = movie_data["budget"].fillna(value=0)

##### 3. Los valores nulos del campo **`release date`** deben eliminarse.

In [15]:
# Eliminación de los valores nulos
movie_data = movie_data.dropna(axis="rows", subset="release_date")

##### 4. De haber fechas, deberán tener el formato **`AAAA-mm-dd`**, además deberán crear la columna **`release_year`** donde extraerán el año de la fecha de estreno.

Usaremo un patrón de regex (regular expression) '^\d{4}-\d{2}-\d{2}$':

* ^: Representa el inicio de la cadena.

* \d{4}: Significa que esperamos exactamente cuatro dígitos. La secuencia \d coincide con cualquier dígito del 0 al 9, y {4} indica que debe haber exactamente cuatro ocurrencias de ese patrón.

* -: Representa el carácter de guion ("-").

* \d{2}: Indica que esperamos exactamente dos dígitos.

* -: Nuevamente, el carácter de guion ("-").

* \d{2}: Otra vez, esperamos exactamente dos dígitos.

* $: Representa el final de la cadena.

In [16]:
# Patrón regex para el formato YYYY-MM-DD
patron = r'^\d{4}-\d{2}-\d{2}$'

# Aplicar filtro utilizando regex
date_format_mask = movie_data["release_date"].str.match(patron)
movie_data[~(movie_data["release_date"].str.match(patron))]

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_id,collection_name,collection_poster_path,collection_backdrop_path,production_company_id,production_company_name
19730,- Written by Ørnås,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Midnight Man,6.0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN
29503,Rune Balot goes to a casino connected to the ...,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Mardock Scramble: The Third Exhaust,7.0,12,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN
35587,Avalanche Sharks tells the story of a bikini ...,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Beware Of Frost Bites,Avalanche Sharks,4.3,22,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN


In [17]:
date_format_mask = movie_data["release_date"].str.match(patron)

In [18]:
collection_backdrop_path_replace = movie_data[~date_format_mask]["budget"] # collection_backdrop_path
date_to_replace = movie_data[~date_format_mask]["id"] # 

In [19]:
movie_data.loc[~date_format_mask, "release_date"] = date_to_replace
movie_data.loc[~date_format_mask, "collection_backdrop_path"] = collection_backdrop_path_replace

In [20]:
movie_data["release_date"] = pd.to_datetime(movie_data["release_date"])

# Conversion de los valores inválidos en valores nulos (NaT).
# movie_data["release_date"] = pd.to_datetime(movie_data["release_date"], errors="coerce")

##### 5. Crear la columna con el retorno de inversión, llamada **`return`** con los campos **`revenue`** y **`budget`**, dividiendo estas dos últimas **`revenue / budget`**, cuando no hay datos disponibles para calcularlo, deberá tomar el valor **`0`**.

In [21]:
# Reemplazo de los valores incosistentes en la columna "budget" por None (Asumimos que nos falta)
# Recordemos que encontramos tres observaciones con datos incosistentes relacionados al "budget"

movie_data.loc[movie_data.index == 19730, "budget"] = None
movie_data.loc[movie_data.index == 29503, "budget"] = None
movie_data.loc[movie_data.index == 35587, "budget"] = None

In [22]:
print(f"Cantidad total de variables/columnas: {movie_data.shape[1]}")

# Evaluacion de las valores faltantes en columnas en los tres registros
movie_data.loc[movie_data[~date_format_mask].index, :].isna().sum(axis=1)

Cantidad total de variables/columnas: 28


19730    15
29503    15
35587    14
dtype: int64

In [23]:
# Son mayores al 50%. Tomamos la decision de eliminarlos
movie_data = movie_data.dropna(subset="budget")

In [24]:
# Parseo de datos de str a float32
movie_data["revenue"] = movie_data["revenue"].astype("float32")
movie_data["budget"] = movie_data["budget"].astype("float32")

In [25]:
# revenue y budget no tienen valores faltantes. Procedemos con el cálculo del retorno (return)
movie_data["return"] = movie_data["revenue"] / movie_data["budget"]

##### 6. Eliminar las columnas que no serán utilizadas, **`video`**,**`imdb_id`**,**`adult`**,**`original_title`**,**`vote_count`**,**`poster_path`** y **`homepage`**

In [26]:
# Estaría correcto eliminar la columna porque no creo que afecten la existencia de Trailer en la decision que un usuario vea una pelicula u otra
movie_data["video"].value_counts() 
# 43 valores repetidos
movie_data["imdb_id"].unique().shape 
# Lo mejor sería eliminar los valores videos recomendados para adultos por se una clase minoritaria
movie_data["adult"].value_counts() 

# Su eliminacion es correcta porque al conservar unicamente title quedará unificado en un solo idioma
movie_data.loc[~(movie_data["original_title"] == movie_data["title"]), ["original_title", "title"]] # Peliculas cuyos "original_title" no coincide con el "title"

# No tiene valore faltantes. Rango de 0.0 a 10.0
movie_data["vote_count"].isna().sum()
np.sort(movie_data["vote_count"].unique()).round(1)

# Tiene 339 valores nulos.
movie_data["poster_path"].isna().sum()
movie_data["homepage"].unique().shape

(7658,)

In [27]:
# Eliminacion de las variables/columnas indicadas
columns_to_remove = ["video", "imdb_id","adult","original_title", "vote_count", "poster_path"]
movie_data = movie_data.drop(columns=columns_to_remove)

In [28]:
# Adicional: Eliminacion 
movie_data = movie_data.drop(['collection_poster_path', 'collection_backdrop_path', 'collection_id', 'production_company_id'] , axis=1)

In [29]:
# Extraer los nombres de los géneros de las peliculas
def extract_genres(text):
    if type(text) == str:
        genres = ", ".join(re.findall(r"'name': '([^']+)'", text))
        return genres.strip()
    else:
        return None

In [30]:
# Aplicamos la funcion de extraccion de los nombres de los generos
movie_data["genres"] = movie_data["genres"].apply(extract_genres)

In [31]:
# Registro duplicados: Elimininamos.
print(f"Hay {movie_data.duplicated().sum()} registros duplicados en el dataset.")

# Eliminamos los registros duplicados
movie_data = movie_data.drop_duplicates()

Hay 13 registros duplicados en el dataset.


In [32]:
def extract_names(text):
    if type(text) == str:
        genres = ", ".join(re.findall(r"'name': '([^']+)'", text))
        return genres.strip()
    else:
        return None

In [33]:
# Aplicamos la funcion que extrae los idiomas
movie_data["spoken_languages"] = movie_data["spoken_languages"].apply(extract_names)

In [34]:
# Extraemos los países donde 
movie_data["production_countries"] = movie_data["production_countries"].apply(extract_names)

In [35]:
# Conversión del id a tipo numérico
movie_data["id"] = movie_data["id"].astype("int32")

In [36]:
# Conversión del popularity a tipo flotante
movie_data["popularity"] = movie_data["popularity"].astype("float32")

``movie_data[["id","popularity"]].groupby("id").transform('mean')`` devuelve un DataFrame con la misma forma que el DataFrame original ``movie_data``, pero con los valores de popularidad reemplazados por la media de la popularidad correspondiente a cada grupo de identificación (id). Esto significa que se calcula la media de la popularidad para cada grupo y se asigna ese valor a todas las filas dentro de ese grupo. El índice y las columnas se mantienen en el DataFrame resultante.

In [37]:
# Nos dimos cuenta que los "id" por cada grupos de registros varía unicamente "popularity" por tanto nos quedaremos con la media de los repetidos
movie_data["popularity"] = movie_data[["id","popularity"]].groupby("id").transform('mean')

In [38]:
print(f'Cantidad de registrados cuyos "id" son duplicados y se va ha eliminar: {movie_data.duplicated(subset="id").sum()} registros.')

# Eliminamos los registros duplicados por id
movie_data = movie_data.drop_duplicates(subset="id")

Cantidad de registrados cuyos "id" son duplicados y se va ha eliminar: 17 registros.


In [39]:
movie_data.head(2)

,budget,genres,homepage,id,original_language,overview,popularity,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,collection_name,production_company_name,return
0,30000000.0,"Animation, Comedy, Family",http://toystory.disney.com/toy-story,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,United States of America,1995-10-30,373554048.0,81.0,English,Released,NaN,Toy Story,7.7,Toy Story Collection,Pixar Animation Studios,12.451801
1,65000000.0,"Adventure, Fantasy, Family",NaN,8844,en,When siblings Judy and Peter discover an encha...,17.015539,United States of America,1995-12-15,262797248.0,104.0,"English, Français",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,NaN,Interscope Communications,4.043035


In [40]:
# Cambiamos el tipo de dato float64 a float32
movie_data.loc[::, "vote_average"] = movie_data["vote_average"].astype("float32")

In [41]:
# Reseteamos el ordenamiento del índice
movie_data = movie_data.reset_index(drop=True)

movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45346 entries, 0 to 45345
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   budget                   45346 non-null  float32       
 1   genres                   45346 non-null  object        
 2   homepage                 7761 non-null   object        
 3   id                       45346 non-null  int32         
 4   original_language        45335 non-null  object        
 5   overview                 44405 non-null  object        
 6   popularity               45346 non-null  float32       
 7   production_countries     45346 non-null  object        
 8   release_date             45346 non-null  datetime64[ns]
 9   revenue                  45346 non-null  float32       
 10  runtime                  45100 non-null  float64       
 11  spoken_languages         45346 non-null  object        
 12  status                   45266 n

## **Queries / Requests de la API**


Deben crear 6 funciones para los endpoints que se consumirán en la API, recuerden que deben tener un decorador por cada una (@app.get(‘/’)).
  
+ def peliculas_mes(mes):
    '''Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron ese mes (nombre del mes, en str, ejemplo 'enero') historicamente'''
    return {'mes':mes, 'cantidad':respuesta}

In [42]:
import locale
# Establecer la configuración regional en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')


def peliculas_mes(mes: str, data: pd.DataFrame):
    # Opcion 1: Crear un diccionario de meses cons respectivos
    # dictionario = {
    #     "enero": "1",
    #     "febrero": "2",
    #     "febrero": "3",
    #     "abril": "4",
    #     "mayo": "5",
    #     "junio": "6",
    #     "julio": "7",
    #     "agosto": "8",
    #     "septiembre": "9",
    #     "octubre": "10",
    #     "noviembre": "11",
    #     "diciembre": "12",
    # }
    # Obtener los nombres de los meses en español
    month_mask = data["release_date"].dt.strftime("%B") == mes.lower()
    count = data[month_mask].shape[0]
    return {
        "mes": mes,
        "cantidad": count
    }

In [43]:
# Ejemplo
peliculas_mes("enero", movie_data)

{'mes': 'enero', 'cantidad': 5909}


+ def peliculas_dia(dia):
    '''Se ingresa el dia y la funcion retorna la cantidad de peliculas que se estrenaron ese dia (de la semana, en str, ejemplo 'lunes') historicamente'''
    return {'dia':dia, 'cantidad':respuesta}

In [44]:
def peliculas_dia(dia: str, data: pd.DataFrame):
    # Obtener los nombres de los días de la semana en español
    day_mask = data["release_date"].dt.strftime("%A") == dia.lower()
    count = data[day_mask].shape[0]
    return {
        "dia": dia,
        "cantidad": count
    }

In [45]:
peliculas_dia('lunes', movie_data)

{'dia': 'lunes', 'cantidad': 3500}

+ def franquicia(franquicia):
    '''Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio'''
    return {'franquicia':franquicia, 'cantidad':respuesta, 'ganancia_total':respuesta, 'ganancia_promedio':respuesta}

In [46]:
def franquicia(franquicia: str, data: pd.DataFrame):
    franchise_name_mask = data["collection_name"].str.lower() == franquicia.lower()
    movie_count = data[franchise_name_mask].shape[0]
    total_revenue = data[franchise_name_mask]["revenue"].sum()
    avg_revenue = data[franchise_name_mask]["revenue"].mean()
    return {
        "franquicia": franquicia,
        "cantidad": movie_count,
        "ganancia_total": total_revenue,
        "ganancia_promedio": avg_revenue
    }

In [47]:
franquicia("Toy Story Collection", movie_data)

{'franquicia': 'Toy Story Collection',
 'cantidad': 3,
 'ganancia_total': 1937890700.0,
 'ganancia_promedio': 645963600.0}

+ def peliculas_pais(pais):
    '''Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo'''
    return {'pais':pais, 'cantidad':respuesta}

In [37]:
def peliculas_pais(pais: str, data: pd.DataFrame):
    country_mask = data["production_countries"].str.lower() == pais.lower()
    movie_count = data[country_mask].shape[0]
    return {
        "pais": pais,
        "cantidad": movie_count
    }


In [39]:
peliculas_pais("United States of America", movie_data)

{'pais': 'United States of America', 'cantidad': 17836}

+ def productoras(productora):
    '''Ingresas la productora, retornando la ganancia total y la cantidad de peliculas que produjeron'''
    return {'productora':productora, 'ganancia_total':respuesta, 'cantidad':respuesta}

In [49]:
movie_data["production_company_name"].unique()

array(['Pixar Animation Studios', 'Interscope Communications',
       'Lancaster Gate', ..., 'CanWest Global Communications',
       'Sine Olivia', 'Yermoliev'], dtype=object)

In [50]:
def productoras(productora: str, data: pd.DataFrame):
    company_name_mask = data["production_company_name"].str.lower() == productora.lower()
    total_revenue  = data[company_name_mask]["revenue"].sum()
    count = data[company_name_mask].shape[0]
    return {
        "productora": productora,
        "ganancia_total": total_revenue,
        "cantidad": count
    }

In [51]:
productoras("Lancaster Gate", movie_data)

{'productora': 'Lancaster Gate', 'ganancia_total': 0.0, 'cantidad': 1}

+ def retorno(pelicula):
    '''Ingresas la pelicula, retornando la inversion, la ganancia, el retorno y el año en el que se lanzo'''
    return {'pelicula':pelicula, 'inversion':respuesta, 'ganacia':respuesta,'retorno':respuesta, 'anio':respuesta}

In [52]:
def retorno(pelicula: str, data: pd.DataFrame):
    movie_title_mask = data["title"].str.lower() == pelicula.lower()
    investment = data[movie_title_mask]["budget"].iloc[0]
    revenue = data[movie_title_mask]["revenue"].iloc[0]
    return_on_investment = data[movie_title_mask]["return"].iloc[0]
    year = data[movie_title_mask]["release_date"].iloc[0].year
    
    return {
        "pelicula": pelicula,
        "inversion": investment,
        "ganancia": revenue,
        "retorno": return_on_investment,
        "anio": year
    }

In [53]:
retorno("Toy Story", movie_data)

{'pelicula': 'Toy Story',
 'inversion': 30000000.0,
 'ganancia': 373554050.0,
 'retorno': 12.451801,
 'anio': 1995}

In [54]:
for i in list(movie_data.columns):
    print(i)

budget
genres
homepage
id
original_language
overview
popularity
production_countries
release_date
revenue
runtime
spoken_languages
status
tagline
title
vote_average
collection_name
production_company_name
return


In [55]:
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45346 entries, 0 to 45345
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   budget                   45346 non-null  float32       
 1   genres                   45346 non-null  object        
 2   homepage                 7761 non-null   object        
 3   id                       45346 non-null  int32         
 4   original_language        45335 non-null  object        
 5   overview                 44405 non-null  object        
 6   popularity               45346 non-null  float32       
 7   production_countries     45346 non-null  object        
 8   release_date             45346 non-null  datetime64[ns]
 9   revenue                  45346 non-null  float32       
 10  runtime                  45100 non-null  float64       
 11  spoken_languages         45346 non-null  object        
 12  status                   45266 n

In [60]:
# Renombramos la columna return
movie_data = movie_data.rename({"return": "return_on_investment"}, axis=1)

In [70]:
# Renombramos la columna id
movie_data = movie_data.rename({"id": "movie_id"}, axis=1)

In [73]:
columns_ordered = [
        "movie_id",
        "budget",
        "genres",
        "homepage",
        "original_language",
        "overview",
        "popularity",
        "production_countries",
        "release_date",
        "revenue",
        "runtime",
        "spoken_languages",
        "status",
        "tagline",
        "title",
        "vote_average",
        "collection_name",
        "production_company_name",
        "return_on_investment"
    ]

In [75]:
# Reordenamos las columas
movie_data = movie_data[columns_ordered]

In [76]:
# Exportamos a un archivo csv
movie_data.to_csv("../api/data/movie_data.csv", index=False)

In [ ]:
# Exportamos a un archivo JSON
movie_data.to_json("../api/data/movie_data.json", orient="records")